# Vergleich: LLM-basierte vs. Pyannote-basierte Sprecherdiarisierung

Dieses Notebook führt zwei Diarisierungs-Pipelines auf denselben Audios aus und speichert pro Datei:
- Transkript (faster-whisper)
- LLM-Diarisierung (textbasiert)
- Pyannote-Diarisierung (audio-basiert) + Mapping auf Whisper-Segmente (nur für menschenlesbare Ausgabe)
- Meta-Informationen inkl. Laufzeiten (Transkription / LLM / Pyannote / Post-Processing)

In [ ]:
### Imports + Konfiguration
import os
from pathlib import Path
import requests
from dotenv import load_dotenv
import time
import json
from datetime import datetime

load_dotenv()

# .env-Variablen (nicht öffentlich) laden + Fehlerbehandlung
VLLM_BASE_URL = os.getenv("VLLM_BASE_URL")
assert VLLM_BASE_URL, "VLLM_BASE_URL ist nicht gesetzt in .env"

VLLM_MODEL = os.getenv("VLLM_MODEL")
assert VLLM_MODEL, "VLLM_MODEL ist nicht gesetzt in .env"

repo_root_env = os.getenv("REPO_ROOT")
assert repo_root_env, "REPO_ROOT ist nicht gesetzt in .env"
REPO_ROOT = Path(repo_root_env).expanduser().resolve()
assert REPO_ROOT.exists(), f"REPO_ROOT existiert nicht: {REPO_ROOT}"

test_audio_env = os.getenv("TEST_AUDIO")
assert test_audio_env, "TEST_AUDIO ist nicht gesetzt in .env"
TEST_AUDIO = (REPO_ROOT / test_audio_env).resolve()
assert TEST_AUDIO.exists(), f"TEST_AUDIO existiert nicht: {TEST_AUDIO}"

# Pfad zu gespeicherten Audios
AUDIO_DIR = REPO_ROOT / "data" / "input_audio"
CONVERTED_DIR = REPO_ROOT / "data" / "normalised_audio"

# Ordner für Ergebnisse
RESULTS_DIR = REPO_ROOT / "results"

## Konfiguration (.env)

Erwartete Variablen:

- `REPO_ROOT` – Pfad zum Repo (z.B. `~/jupyter/diarization-benchmark`)
- `TEST_AUDIO` – relativer Pfad zu einer Testdatei innerhalb des Repos
- `VLLM_BASE_URL` – vLLM OpenAI-API Endpoint (typisch inkl. `/v1`, z.B. `http://127.0.0.1:8003/v1`)
- `VLLM_MODEL` – served model name, z.B. google/medgemma-27b-it
- `HUGGINGFACE_TOKEN` – Token für pyannote Modelle
- `PYANNOTE_MODEL_ID` – z.B. `pyannote/speaker-diarization-community-1`

Ordner:
- Input: `data/input_audio/`
- Normalisierte WAVs: `data/normalised_audio/`
- Outputs: `results/<datei>/`

## Audio-Normalisierung (ffmpeg)

Für Pyannote verwenden wir einheitliche Audios: **WAV, 16 kHz, mono**.  
Die Normalisierung wird gecached (existierende konvertierte Dateien werden wiederverwendet).

In [ ]:
### Audio-Normalisierung via ffmpeg, alles konvertieren zu WAV 16khz Mono -> Wichtig für Pyannote
import subprocess
from pathlib import Path

def ensure_wav_16k_mono(input_path: str | Path, out_dir: str | Path) -> Path:
    input_path = Path(input_path)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    out_path = out_dir / f"{input_path.stem}_16k_mono.wav"

    # Reuse, wenn schon vorhanden
    if out_path.exists() and out_path.stat().st_size > 0:
        return out_path

    cmd = [
        "ffmpeg", "-y",
        "-i", str(input_path),
        "-ac", "1",        # mono
        "-ar", "16000",    # 16kHz
        "-vn",             # no video
        str(out_path),
    ]
    res = subprocess.run(cmd, capture_output=True, text=True)
    if res.returncode != 0:
        raise RuntimeError(f"ffmpeg failed:\n{res.stderr}")
    return out_path

In [ ]:
### Konvertierungs-Test
# Testaudio mit normalisierter Version überschreiben
conversion_test = False
if conversion_test:
    TEST_AUDIO = ensure_wav_16k_mono(TEST_AUDIO, CONVERTED_DIR)

    print("WAV:", TEST_AUDIO)
    print("Exists:", TEST_AUDIO.exists(), "Size:", TEST_AUDIO.stat().st_size)

## Transkribierung mit faster-whisper (lokal)

Wir transkribieren jedes Audio genau einmal (auf der normalisierten WAV), um:
- ein gemeinsames Transkript für LLM und Vergleichsausgaben zu haben
- Whisper-Segmente als Zeitbasis für das Mapping der Pyannote-Turns zu nutzen

In [ ]:
### Whisper-Config
from faster_whisper import WhisperModel

WHISPER_MODEL_SIZE = os.getenv("WHISPER_MODEL_SIZE", "small") # Kleines Modell für Tests
# WHISPER_MODEL_SIZE = os.getenv("WHISPER_MODEL_SIZE", "large-v3") # Großes Modell für Prod?
WHISPER_DEVICE = os.getenv("WHISPER_DEVICE", "cpu")  # Für unseren Test auf CPU laufen lassen -> langsamer
# WHISPER_DEVICE = os.getenv("WHISPER_DEVICE", "cuda") # Wenn verfügbar: Auf GPU laufen lassen -> schneller -> Aber: Konfig-Anpassungen notwendig!
WHISPER_COMPUTE_TYPE = os.getenv("WHISPER_COMPUTE_TYPE", "int8")  # cpu: int8 gut

whisper_model = WhisperModel(WHISPER_MODEL_SIZE, device=WHISPER_DEVICE, compute_type=WHISPER_COMPUTE_TYPE)

In [ ]:
### Transkription
def transcribe_faster_whisper(audio_path: str, language: str | None = None):
    segments_iter, info = whisper_model.transcribe(
        audio_path,
        language=language,
        vad_filter=True,
        word_timestamps=False,  # später ggf. True, falls wir word-level brauchen
    )
    segments = []
    texts = []
    for seg in segments_iter:
        txt = seg.text.strip()
        segments.append({"start": float(seg.start), "end": float(seg.end), "text": txt})
        texts.append(txt)
    transcript = "\n".join(texts).strip()
    return transcript, segments, info

In [ ]:
### Whisper-Test anhand der Test-Audio. Aktivieren -> True setzen
whisper_test = False
if whisper_test:
    t, segs, info = transcribe_faster_whisper(TEST_AUDIO, language="de")
    print("Language:", info.language, "Prob:", info.language_probability)
    print("Transcript:\n", t[:1000])
    print("First segments:", segs[:3])

## Pipeline A: Whisper → LLM-Diarisierung (textbasiert)

Eingabe: reines Transkript  
Ausgabe: `[Sprecher 1]: ... [Sprecher 2]: ...`

Diese Pipeline nutzt ausschließlich Text (keine Audioinformation). Prompts sind in `diarize_with_llm` definiert.

In [ ]:
### vLLM-Client

def chat_vllm(messages, model=VLLM_MODEL, temperature=0.0, max_tokens=200, timeout=600):
    url = f"{VLLM_BASE_URL}/chat/completions"
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    r = requests.post(url, json=payload, timeout=timeout)
    r.raise_for_status()
    data = r.json()

    content = data["choices"][0]["message"].get("content", None)
    if content is None:
        raise RuntimeError(f"LLM returned no content. Full response: {json.dumps(data)[:2000]}")
    return content


In [ ]:
# Kurzer LLM-Test: Bei Bedarf auf True setzen
llm_test = False

if llm_test:
    print(chat_vllm([{"role":"user","content":"Antworte nur mit OK"}]))

In [ ]:
### Diarisierungs-Logik inklusive Prompts
def diarize_with_llm(transcript: str):

    system_prompt = (
        "You are a professional conversation diarization engine. "
        "Assign speaker labels logically solely based on the text. "
        "Output only the diarized transcript."
    )

    user_prompt = (
        "Add speaker labels to the following transcript. "
        "Use '[Sprecher 1]', '[Sprecher 2]', ... for the different speakers. "
        "Check thoroughly for every sentence if the assignment to the speaker is contextually and logically plausible. "
        "Produce a readable dialogue format between Speakers.\n\n"
        f"{transcript}"
    )


    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    return chat_vllm(messages, temperature=0.0, max_tokens=2000)

In [ ]:
# Kombinierter Test: Whisper-Diarisierung -> LLM-Diarisierung anhand der Test-Audio, True setzen für Test
whisper_llm_test = False
if whisper_llm_test:
    print("Starte Transkription mit faster-whisper")
    t0 = time.time()
    transcript, segs, info = transcribe_faster_whisper(TEST_AUDIO, language="de")
    t1 = time.time()
    print(f"Whisper-Transkription abgeschlossen in ({t1 - t0:.2f}s)")
    print("LLM-Diarisierung gestartet")
    t2 = time.time()
    diarized = diarize_with_llm(transcript)
    t3 = time.time()
    print(f"LLM-Diarisierung abgeschlossen in ({t3 - t2:.2f}s)")
    print(diarized)

## Pipeline B: Pyannote-Diarisierung (audio-basiert)

Pyannote liefert alleinig **Speaker-Turns** (wer spricht wann).  
Damit das in eine menschenlesbare Gesamt-Ausgabe überführt wird, benötigen wir **Post-Processing**:
- Speaker-Turns auf Whisper-Segmente mappen (Overlap-Heuristik) als Kern-Prozess
- Weitere nicht-rechenintensive kosmetische Nachbearbeitungen und Formatierungen für Vergleichbarkeit mit LLM-Ausgabe

**Limitation:** Segment-basiertes Mapping kann kurze Einwürfe in langen Segmenten “verschlucken” und hat Schwierigkeiten mit ähnlichen Stimmen der Sprecher.

In [ ]:
### Pyannote-Setup (CPU-only)
import torch
from pyannote.audio import Pipeline

HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
assert HF_TOKEN, "HUGGINGFACE_TOKEN fehlt in .env" # Fehlerbehandlung

PYANNOTE_MODEL_ID = os.getenv("PYANNOTE_MODEL_ID")
assert PYANNOTE_MODEL_ID, "PYANNOTE_MODEL_ID fehlt in .env"

pyannote_pipeline = Pipeline.from_pretrained(PYANNOTE_MODEL_ID, token=HF_TOKEN)

# CPU erzwingen
pyannote_pipeline.to(torch.device("cpu"))

In [ ]:
### Pyannote starten
def run_pyannote(wav_path: Path):
    
    diarization = pyannote_pipeline(str(wav_path))

    return diarization

In [ ]:
### Minimales Pyannote Post-Processing (Mapping auf die Whisper-Segmente)
import re

### Pipeline: ensure_wav_16k_mono -> transcribe_faster_whisper -> run_pyannote -> label_whisper_segments -> normalize_speakers 
### (für Lesbarkeit, optional) -> merge_same_speakers (für Lesbarkeit, optional) -> format_dialogue (optional?)

def pyannote_turns(diarization):
    # diarization kann Annotation sein oder ein Objekt mit .speaker_diarization (Letzteres in unserem Fall)
    ann = getattr(diarization, "speaker_diarization", diarization)
    turns = []
    for turn, _, speaker in ann.itertracks(yield_label=True):
        turns.append((float(turn.start), float(turn.end), str(speaker)))
    turns.sort(key=lambda x: (x[0], x[1]))
    return turns

def overlap(a0, a1, b0, b1):
    return max(0.0, min(a1, b1) - max(a0, b0))

# Haupt-Funktion zum Mappen
def label_whisper_segments(segments, diarization, min_overlap_s=0.05):

    turns = pyannote_turns(diarization)
    labeled = []

    for seg in segments:
        s0, s1 = float(seg["start"]), float(seg["end"])
        best_spk, best_ol = "UNK", 0.0

        for t0, t1, spk in turns:
            if t1 <= s0:
                continue
            if t0 >= s1:
                break
            ol = overlap(s0, s1, t0, t1)
            if ol > best_ol:
                best_ol, best_spk = ol, spk

        if best_ol < min_overlap_s:
            best_spk = "UNK"

        labeled.append({
            "start": s0, "end": s1,              # intern behalten
            "speaker_raw": best_spk,
            "text": (seg.get("text") or "").strip()
        })

    return labeled

### Kosmetische Funktion zur Vergleichbarkeit mit LLM-Ausgabe SPEAKER_00 -> Sprecher 1
def normalize_speakers(labeled):
    mapping = {}
    n = 1
    for x in labeled:
        raw = x["speaker_raw"]
        if raw != "UNK" and raw not in mapping:
            mapping[raw] = f"[Sprecher {n}]"
            n += 1
    for x in labeled:
        x["speaker"] = mapping.get(x["speaker_raw"], "UNK")
    return labeled

def merge_same_speaker(labeled, max_gap_s=0.8):
    # nur Lesbarkeit: aufeinanderfolgende Segmente des gleichen Speakers zusammenziehen
    # (Gar kein Heuristik-Merging: max_gap_s=0 setzen)
    out = []
    for seg in labeled:
        if not seg["text"]:
            continue
        if not out:
            out.append(dict(seg))
            continue

        prev = out[-1]
        gap = seg["start"] - prev["end"]
        if seg["speaker"] == prev["speaker"] and gap <= max_gap_s:
            prev["text"] = (prev["text"].rstrip() + " " + seg["text"].lstrip()).strip()
            prev["end"] = seg["end"]
        else:
            out.append(dict(seg))
    return out

def format_dialogue(utterances):
    lines = []
    for u in utterances:
        txt = re.sub(r"\s+", " ", u["text"]).strip()
        if txt:
            lines.append(f"{u['speaker']}: {txt}")
    return "\n".join(lines)

In [ ]:
### Pyannote-Test mit vorheriger Transkribierung (für Segmente), bei Bedarf auf True setzen
pyannote_test = False
if pyannote_test:
    print("Normalisiere Datei für Pyannote (wav-Datei mit 16k mono)")
    wav_path = ensure_wav_16k_mono(TEST_AUDIO, CONVERTED_DIR)
    print("Normalisierung abgeschlossen")
    print("Starte Transkription mit faster-whisper")
    t0 = time.time()
    transcript, segs, info = transcribe_faster_whisper(wav_path, language="de")
    t1 = time.time()    
    print(f"Whisper-Transkription abgeschlossen in ({t1 - t0:.2f}s)")
    print("Pyannote gestartet...")
    t2 = time.time()
    diarization = run_pyannote(wav_path)
    t3 = time.time()
    print(f"Pyannote abgeschlossen in ({t3 - t2:.2f}s)")
    print("Text-Aufbereitung begonnen")
    t4 = time.time()
    # alle Schritte als eigene Variable speichern für Debugging
    labeled = label_whisper_segments(segs, diarization, min_overlap_s=0.05) # 0.05 verbreiteter Default-Wert
    labeled2 = normalize_speakers(labeled)
    labeled3 = merge_same_speaker(labeled2, max_gap_s=0.8)
    labeled4 = format_dialogue(labeled3)
    t5 = time.time()
    print(f"Nachbearbeitung abgeschlossen in ({t5-t4:.2f}s)")
    print(f"Gesamt-Pipeline (Pyannote + Nachbereitung) abgeschlossen in ({t5-t2:.2f}s)")

    print("End-Ausgabe:\n" + labeled4)

## Batch-Run (alle Audios in `input_audio`)

Führt den Vergleich (Pyannote/LLM) für mehrere Dateien auf einmal aus und speichert Metadaten zur Auswertung.

Für jede Datei wird ein eigener Ordner unter `results/` erzeugt mit:
- `transcript.txt` als von Whisper erzeugtes Transkript zur Audio
- `llm_diarized.txt` als der mittels LLM diarisierte Text
- `pyannote_diarized.txt` als der mittels pyannote diarisierte Text
- `meta.json` (Laufzeiten, Modelle, Status)
- `whisper_segments.json` und `whisper_info.json` mit Infos zur Transkription und den Segmenten, hauptsächlich zu Debug-Zwecken

Zusätzlich wird `results/batch_summary.json` als globale Datei, die sämtliche Meta-Daten enthält, erstellt (wird bei Re-Run überschrieben).

In [ ]:
### Input-Audios vorbereiten und Hilfsfunktionen für das Speichern von Ergebnissen

RESULTS_DIR.mkdir(parents=True, exist_ok=True)
BATCH_AUDIO_DIR = REPO_ROOT / "data" / "input_audio"

def save_text(path: Path, content: str):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(content or "", encoding="utf-8")

def save_json(path: Path, obj: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2, ensure_ascii=False), encoding="utf-8")

def list_audio_files(folder: Path) -> list[Path]:
    exts = {".wav", ".mp3", ".m4a", ".flac", ".ogg"}
    files = [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]
    return sorted(files)

audio_files = list_audio_files(BATCH_AUDIO_DIR)
print("Found files:", len(audio_files))


In [ ]:
### Haupt-Pipeline
### pro Datei: wav-Konvertierung -> fasterwhisper -> LLM-Diarisierung / Pyannote + Post-Processing
### speichert sämtliche Laufzeiten und Metainformationen
def run_compare_on_file(audio_path: Path) -> dict:
    
    out_dir = RESULTS_DIR / audio_path.stem
    out_dir.mkdir(parents=True, exist_ok=True)
    
    t0_total = time.time()

    # 1) Audio normalisieren (einmal, WAV 16k Mono)
    t0_norm = time.time()
    wav_path = ensure_wav_16k_mono(audio_path, CONVERTED_DIR)
    t1_norm = time.time()

    # 2) Audio transkribieren (einmal, auf derselben WAV wie Pyannote)
    t0_transc = time.time()
    transcript, segs, info = transcribe_faster_whisper(str(wav_path), language="de")
    t1_transc = time.time()

    save_text(out_dir / "transcript.txt", transcript)
    save_json(out_dir / "whisper_segments.json", segs)  # zu Debug-Zwecken
    save_json(out_dir / "whisper_info.json", {
        "language": getattr(info, "language", None),
        "language_probability": float(getattr(info, "language_probability", 0.0) or 0.0),
        "segments_count": len(segs),
        "transcript_chars": len(transcript),
        "transcript_words": len(transcript.split()),
    })

    # 3) LLM-Diarisierung
    llm_ok, llm_err = True, None
    diarized_llm = ""
    t0_llm = time.time()
    try:
        llm_input = transcript
        diarized_llm = diarize_with_llm(llm_input)
    except Exception as e:
        llm_ok, llm_err = False, repr(e)
    t1_llm = time.time()

    save_text(out_dir / "llm_diarized.txt", diarized_llm)

     # 4) Pyannote-Diarisierung + Post-Processing (segment-basiert)
    py_ok, py_err = True, None
    diarized_py = ""

    t0_py = time.time()
    try:
        diarization = run_pyannote(wav_path)
    except Exception as e:
        py_ok, py_err = False, repr(e)
        diarization = None
    t1_py = time.time()

    t0_post = time.time()
    try:
        if diarization is not None:
            labeled = label_whisper_segments(segs, diarization, min_overlap_s=0.05)
            labeled = normalize_speakers(labeled)
            labeled = merge_same_speaker(labeled, max_gap_s=0.8) 
            diarized_py = format_dialogue(labeled)
    except Exception as e:
        py_ok, py_err = False, (py_err or "") + " | postproc: " + repr(e)
    t1_post = time.time()

    save_text(out_dir / "pyannote_diarized.txt", diarized_py)

    t1_total = time.time()

    # Meta-Informationen speichern, Zeiten auf 2 Nachkommastellen runden
    meta = {
        "timestamp": datetime.now().isoformat(),
        "audio_path": str(audio_path),
        "wav_path": str(wav_path),
        "models": {
            "whisper_model_size": WHISPER_MODEL_SIZE,
            "whisper_device": WHISPER_DEVICE,
            "whisper_compute_type": WHISPER_COMPUTE_TYPE,
            "llm_model": VLLM_MODEL,
            "pyannote_model_id": PYANNOTE_MODEL_ID,
        },
        "status": {
            "llm_ok": llm_ok,
            "llm_error": llm_err,
            "pyannote_ok": py_ok,
            "pyannote_error": py_err,
        },
        "counts": {
            "whisper_segments": len(segs),
            "transcript_chars": len(transcript),
            "transcript_words": len(transcript.split()),
        },
        "seconds": {
            "total": round(t1_total - t0_total, 2),
            "normalize": round(t1_norm - t0_norm, 2),
            "transcribe": round(t1_transc - t0_transc, 2),
            "llm": round(t1_llm - t0_llm, 2),
            "pyannote": round(t1_py - t0_py, 2),
            "postproc": round(t1_post - t0_post, 2),
        },
        "params": {
            "min_overlap_s": 0.05,
            "merge_max_gap_s": 0.8,
        }
    }

    save_json(out_dir / "meta.json", meta)
    return meta

In [ ]:
### Ausführen der Pipeline für alle Audios und Anlegen einer globalen Summary-Datei

metas = []
failures = []

for i, f in enumerate(audio_files, 1):
    print(f"\n[{i}/{len(audio_files)}] {f.name}")
    try:
        meta = run_compare_on_file(f)
        metas.append(meta)

        # Kontroll-Ausgaben
        print(
            "  total:", meta["seconds"]["total"], "s",
            "| transc:", meta["seconds"]["transcribe"], "s",
            "| llm:", meta["seconds"]["llm"], "s",
            "| py:", meta["seconds"]["pyannote"], "s",
            "| post:", meta["seconds"]["postproc"], "s",
            "| ok:", meta["status"]["llm_ok"], "/", meta["status"]["pyannote_ok"]
        )
    
    # Fehlschläge direkt tracken zum Debuggen
    except Exception as e:
        failures.append({"file": str(f), "error": repr(e)})
        print(" FAIL:", repr(e))

### Alles als Gesamt-Zusammenfassung in JSON speichern für Auswertungszwecke
save_json(RESULTS_DIR / "batch_summary.json", {
    "timestamp": datetime.now().isoformat(),
    "files_total": len(audio_files),
    "metas_count": len(metas),
    "failures_count": len(failures),
    "failures": failures,
    "metas": metas,
})

print("Done. Summary:", RESULTS_DIR / "batch_summary.json")